In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

In [4]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
       text = f.read()

chars = sorted(set(text))
vocab_size = len(chars)

In [5]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l:  ''.join([int_to_string[c] for c in l])

data = torch.tensor(encode(text), dtype=torch.long)
data


tensor([80,  1,  1,  ..., 29, 67, 57])

In [6]:
n = int(0.8 * len(data))

train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[11,  3,  0,  0,  3, 26, 74, 73],
        [ 1, 66, 78,  1, 55, 68, 67, 58],
        [ 1, 67, 68,  1, 67, 58, 58, 57],
        [ 1, 43, 68, 71, 56, 58, 71, 58]])
targets:
tensor([[ 3,  0,  0,  3, 26, 74, 73,  1],
        [66, 78,  1, 55, 68, 67, 58, 72],
        [67, 68,  1, 67, 58, 58, 57,  1],
        [43, 68, 71, 56, 58, 71, 58, 71]])


In [12]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



5-k')XZ(S,ZjkBV1z_BAx7X&mNCiAy yNIZ035-s32X&7﻿hnr0acsW6TF8Q 
sTHJ[Mo2wI"hg' H4K?
3L)?'&E
?I96CujFzuMGKk;ly1J&Chg!)(sNV
za!pwv?mmI,4ZO﻿Y35GPe?hY[﻿R_iTns8Xbm[-PTE MZ ?
(l9]ir8L0Pu[M)VwIa3-[M*mtX[Y?-t3kVtu8qIIGXI'8VIp!80y&7gMN:ua32t6?h
deLQ6]YnxAm-gLvuYVeipy2_y2y3mljPVtE]nIXcx"xl[GxTuVm:IvS],9?4Enfk5)rrRkT TqZ6:"22sc&&Mx*y)8BChTNVpvQ_ )E6tS&E﻿[,;s?mtt]!h_n69Q:,V
﻿(SZ.4&J'nrQR"&nA*oYtEc5teIa)8),!vYm﻿UwarwIdV47﻿Jmt2r
"MPM?O'FD&1R3!j4669OsM4U!I:U'v&EH4 ?BAYkV!pWD3&z6.X9vBC;nrSehQ6IXQgMaaDfbnfYThn_[M8q


In [8]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [16]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.682, val loss: 2.656
step: 250, train loss: 2.648, val loss: 2.674
step: 500, train loss: 2.657, val loss: 2.693
step: 750, train loss: 2.644, val loss: 2.657
2.4928836822509766


In [10]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


tz: cyF'7jEWNangC-,p1!8VcWajdLHZd80)Di&&0x5I0nO&SY,TkmzEeK,qmzqmh?z.
I['-uIFFBgh? ﻿d,Zw:z﻿p*FbIbAlzAyy
!cW9?49Oc4F'Agkm[﻿EGl4RMSgQfo!mY:0kmve:6n9TzUGX':7uQUHSalI!C&Y!C3-d-bHF,W]SI'4p-U,4t-4Mkn-8,bAR﻿u!hc[S o75yz*;.r31*9T'eKl"h?t'&1Jt1?utehdTmj6R9UJLjIS_UF-yNe&&8dl4su7QsSaD3FPMn_5QeKwy) ?Y6n4F'eT6hnq-CU0:*]wOWWXr3o!cMSOBdH!d_.?RYV.C)mFEs3up,!﻿OQ
QUrLYOno5t-jx]s.bGa7?q[xtRo;adwA'I8Q;e&R&f8
NsGa ZUZSjao[!TOK
Ie3vh?5e;Mi")0M: LRTq0"q;ty
P﻿dHpI'Q(,bA"﻿3lsI8uIC9?1Em7MxrenX'ZocB'IM7)[X.AasZRjMs0bpYgPj!
